In [6]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torchvision
from torch.nn import functional 
from pydub import AudioSegment
import librosa

In [3]:
# Import EfficientNet model -- Using b0 for speed 
efnet_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
efnet_model = torchvision.models.efficientnet_b0(weights = efnet_weights)

In [15]:
MAPPING = {
    "acoustic" : 0,
    "blues" : 1,
    "classical" : 2,
    "country" : 3,
    "dance" : 4,
    "edm" : 5,
    "hip-hop" : 6,
    "indie" : 7,
    "jazz" : 8,
    "pop" : 9,
    "rock" : 10,
    "world-music" : 11
}

IMG_FOLDER = "spectrogram"

genres = os.listdir(IMG_FOLDER)

#! Using zeros instead of empty because empty needs dtypes
images = np.zeros(shape = (1195, 4, 65, 86))
labels = np.zeros(shape = 1195)

idx = 0
added = []
for root, directories, contents in os.walk(IMG_FOLDER):
    if root == IMG_FOLDER: continue
    genre = root.split('/')[1]
    if genre in MAPPING:
        for img_path in contents:
            image = plt.imread(root + "/" + img_path)
            ## Convert to tensor and check the shape
            spec = torch.from_numpy(image)

            ## Move third dimension (color channels) to the first dimension
            spec2 = torch.movedim(spec, source=2, destination=0)

            ## Add an empty first dimension, putting our tensor into the standard format
            spec_final = torch.unsqueeze(spec2, dim=0)
            
            ##! Why /6 ?
            spec_resized = functional.interpolate(spec_final, size = (65,86)) ### 389 / 6 = 65 and 515 / 6 = 86
            
            images[idx] = spec_resized
            labels[idx] = MAPPING[genre]
            idx += 1


In [16]:
## Train-test split
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(images, labels, test_size=0.25, random_state=5)

In [ ]:
## Neural network with transfered Efnet